In [1]:
import re
import pandas as pd

In [2]:
df = pd.read_csv('..\datasets\dataset-maritaca-teste1.csv')
df

,sentenca,categoria_maritaka
0,"Atendimento bom, lugar pequeno, porém aconcheg...","ambiente, geral"
1,Lugar agradável atendimento não é muito bom po...,"serviço, comida"
2,"Pra quem gosta de uma vida noturna, este é um ...","comida, bebida, serviço, ambiente, localização"
3,"Para quem já foi em uma boa churrascaria, a Bú...","comida, geral"
4,Localizado em um ótimo shopping em Manaus.,localização
...,...,...
3995,"Local bem bacana, ótimo pra desestressar e apr...",NaN
3996,Atendimento de um rapaz atencioso e educado Sa...,NaN
3997,"Tem comida, muuuuita comida!",NaN
3998,Muito bom o atendimento do Emerson local muito...,NaN


In [3]:
num_linhas_nulas = df['categoria_maritaka'].isnull().sum()
num_linhas_nulas

1149

In [4]:
sentenca = 'Não oferece o melhor serviço nem o melhor preço mas também fornece uma comida muito boa.'
linhas_correspondentes = df.loc[df['sentenca'] == sentenca]
linhas_correspondentes

,sentenca,categoria_maritaka
2836,Não oferece o melhor serviço nem o melhor preç...,"<frase>serviço, comida, preço"


In [5]:
# Abrir o arquivo de entrada e de saída
with open('logs/maritaca-teste1-chaves.log', 'r', encoding='utf-8') as infile, open('logs/maritaca-teste1-processado.log', 'w', encoding='utf-8') as outfile:
    current_sentence = None
    current_response = None

    for line in infile:
        if line.startswith('INFO:root:SENTENÇA: {'):
            # Se encontramos uma nova sentença, processamos a linha
            if current_sentence is not None and current_response is not None:
                # Escrevemos a sentença e a resposta anteriores no arquivo de saída
                outfile.write(f"{current_sentence.strip()}{current_response.strip()}\n")
            
            # Inicializamos a nova sentença com a linha atual
            current_sentence = line.strip()
            current_response = ""

        elif line.startswith('INFO:root:RESPONSE: {'):
            # Se encontramos uma nova resposta, começamos a coletar suas linhas
            current_response += line.strip()

        elif line.startswith('INFO:root:CHAT:'):
            continue
            # Se encontramos uma linha de chat, ignoramos

        else:
            # Se não é uma linha relevante, adicionamos ao conteúdo da resposta atual
            if current_response is not None:
                current_response += line.strip()

    # Escrevemos a última sentença e resposta no arquivo de saída
    if current_sentence is not None and current_response is not None:
        outfile.write(f"{current_sentence.strip()}{current_response.strip()}\n")



In [6]:
log_file_path = 'logs/maritaca-teste1-processado.log'

sentencas = []
responses = []

# Expressões regulares para encontrar sentenças e respostas entre chaves {}
sentenca_pattern = re.compile(r'SENTENÇA: {(.+?)}')
response_pattern = re.compile(r'RESPONSE: {(.*?)}', re.DOTALL)  # Use re.DOTALL to match across multiple lines

with open(log_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    multiline_response = ""

    for line in lines:
        line = line.strip()
        
        # Debugging output: print each line to understand the parsing process
        print("Processing line:", line)
        
        # Procurar por sentenças
        sentenca_match = sentenca_pattern.search(line)
        if sentenca_match:
            sentenca = sentenca_match.group(1)
            sentencas.append(sentenca)
            
            # Debugging output: print the matched sentence
            print("Found sentence:", sentenca)
            
            # Se tiver uma resposta acumulada, adiciona à lista de respostas
            if multiline_response:
                responses.append(multiline_response.strip())
                multiline_response = ""
        
        # Procurar por respostas
        response_match = response_pattern.search(line)
        if response_match:
            response = response_match.group(1).strip()
            
            # Debugging output: print the matched response
            print("Found response:", response)
            
            # Se a resposta estiver em várias linhas, acumula até encontrar a próxima sentença
            if response:
                multiline_response += response
            else:
                # Se a resposta está vazia, indica que é uma resposta multi-linha
                multiline_response = ""
    
    # Verifica se há uma resposta pendente no final do arquivo
    if multiline_response:
        responses.append(multiline_response.strip())

# Verificar o tamanho das listas após o processamento
print("Tamanho de Sentenças:", len(sentencas))
print("Tamanho de Responses:", len(responses))

Processing line: INFO:root:SENTENÇA: {Atendimento bom, lugar pequeno, porém aconchegante onde podes escolher entra área externa ou interna.}INFO:root:RESPONSE: {ambiente, geral}
Found sentence: Atendimento bom, lugar pequeno, porém aconchegante onde podes escolher entra área externa ou interna.
Found response: ambiente, geral
Processing line: INFO:root:SENTENÇA: {Lugar agradável atendimento não é muito bom por alguns garçons do segundo piso voltaria pela pizza}INFO:root:RESPONSE: {serviço, comida}
Found sentence: Lugar agradável atendimento não é muito bom por alguns garçons do segundo piso voltaria pela pizza
Found response: serviço, comida
Processing line: INFO:root:SENTENÇA: {Pra quem gosta de uma vida noturna, este é um bar histórico e com uma tradição forte para os moradores de Manaus, tem um sanduíche de pernil que é fora de serie, som, petiscos, um povo muito hospitaleiro e bem no Centro da cidade!}INFO:root:RESPONSE: {comida, bebida, serviço, ambiente, localização}
Found senten

In [7]:
data = {'sentenca': sentencas, 'categoria_maritaka_logs': responses}
df_logs = pd.DataFrame(data)
df_logs

,sentenca,categoria_maritaka_logs
0,"Atendimento bom, lugar pequeno, porém aconcheg...","ambiente, geral"
1,Lugar agradável atendimento não é muito bom po...,"serviço, comida"
2,"Pra quem gosta de uma vida noturna, este é um ...","comida, bebida, serviço, ambiente, localização"
3,"Para quem já foi em uma boa churrascaria, a Bú...","comida, geral"
4,Localizado em um ótimo shopping em Manaus.,localização
...,...,...
2832,Comida deliciosa e preço muito bom.,"comida, preço"
2833,Aí o rapaz me informou que deveria falar com o...,"serviço, preço, geral"
2834,Boa comida por um preço justo,"comida, preço"
2835,Preço bem diferente dos restaurantes de São Pa...,preço


In [8]:
df_primeiras_linhas = df.head(2837)

# Adicione uma nova coluna 'categoria_maritaka_logs' em df_primeiras_linhas
df_primeiras_linhas['categoria_maritaka_logs'] = df_logs['categoria_maritaka_logs']

# Crie um novo DataFrame df_merged concatenando df_primeiras_linhas com o restante do DataFrame df
df_resto = df.iloc[2837:]  # Pegue o restante das linhas de df após as primeiras 2837 linhas
df_2837 = pd.concat([df_primeiras_linhas, df_resto])

df_2837 = df_2837.drop(['categoria_maritaka'], axis=1)
df_2837 = df_2837.rename(columns={'categoria_maritaka_logs': 'categoria_maritaka'})
df_2837.to_csv(f'../datasets/dataset-maritaca-teste1.csv', index=False)
df_2837

C:\Users\José Neto\AppData\Local\Temp\ipykernel_29064\1776969183.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_primeiras_linhas['categoria_maritaka_logs'] = df_logs['categoria_maritaka_logs']


,sentenca,categoria_maritaka
0,"Atendimento bom, lugar pequeno, porém aconcheg...","ambiente, geral"
1,Lugar agradável atendimento não é muito bom po...,"serviço, comida"
2,"Pra quem gosta de uma vida noturna, este é um ...","comida, bebida, serviço, ambiente, localização"
3,"Para quem já foi em uma boa churrascaria, a Bú...","comida, geral"
4,Localizado em um ótimo shopping em Manaus.,localização
...,...,...
3995,"Local bem bacana, ótimo pra desestressar e apr...",NaN
3996,Atendimento de um rapaz atencioso e educado Sa...,NaN
3997,"Tem comida, muuuuita comida!",NaN
3998,Muito bom o atendimento do Emerson local muito...,NaN
